<a href="https://colab.research.google.com/github/TungLam111/recommend-fuck-minathon-shit/blob/main/minathon_2021_rec_content_collaberative_shit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/hotel_reviews.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   address               499 non-null    object
 1   categories            499 non-null    object
 2   city                  499 non-null    object
 3   country               499 non-null    object
 4   name                  499 non-null    object
 5   postalCode            499 non-null    int64 
 6   province              499 non-null    object
 7   reviews.rating        499 non-null    int64 
 8   reviews.text          499 non-null    object
 9   reviews.title         490 non-null    object
 10  reviews.userCity      499 non-null    object
 11  reviews.username      499 non-null    object
 12  reviews.userProvince  499 non-null    object
dtypes: int64(2), object(11)
memory usage: 50.8+ KB


In [ ]:
df.groupby(["reviews.userCity"]).size()

reviews.userCity
Albany               1
Amarillo             1
Anchorage            1
Ann Arbor            1
Athens               9
                    ..
Wirral               1
houston              1
kehaulanifrommaui    1
maryland             1
ny                   1
Length: 84, dtype: int64

In [ ]:
df = df.drop(["reviews.date", "reviews.dateAdded", "reviews.id", "reviews.doRecommend"], axis = 1)

In [ ]:
df = df.drop(["latitude", "longitude"], axis = 1)

In [ ]:
df['reviews.userCity'].fillna("Other", inplace=True)

In [ ]:
df['reviews.userProvince'].fillna("Other", inplace=True)

In [ ]:
df.head()

,address,categories,city,country,name,postalCode,province,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,4,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,Other,Russ (kent),Other
1,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,Other,A Traveler,Other
2,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,Other,Maud,Other
3,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,We stayed here for four nights in October. The...,Good location on the Lido.,Other,Julie,Other
4,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,We stayed here for four nights in October. The...,������ ���������������,Other,sungchul,Other


In [ ]:
df['reviews.userProvince'].fillna("Other", inplace=True)

In [ ]:
df['reviews.title'].fillna("Other", inplace=True)

In [ ]:
A = []
hihi = '�'
for i in range(499):
    print(df.iloc[i]["reviews.title"])
    if hihi in df.iloc[i]["reviews.title"] :
        A.append(i)

Good location away from the crouds
Great hotel with Jacuzzi bath!
Lugnt l��ge
Good location on the Lido.
������ ���������������
Very nice hotel
Lovely view out onto the lagoon. Excellent view.
Lovely view out onto the lagoon. Excellent view.
G��nstige Lage
Ro og hygge
Tr��s bon h��tel calme et agr��able.
It was ok hotel is nice from in and out but room
Sehr angenehmes Hotel !
Tip top
Lugnet p�� Lido
Lugnet p�� Lido
Nice hotel with very friendly and helpful staff
Guter Ausgangspunkt f��r Ausfl��ge
���������������.
Hotellihuone oli pettymys
Dungeons, Drain Flies and Centipedes
Excellent hotel with good access to Venice
Sch��ne Lage, ruhige Gegend
Lovely stay - second time staying at this hotel.
Tolles Hotel f��r einen St��dtetrip nach Venedig
A good Hotel on the Lido - Quiet location.
A good Hotel on the Lido - Quiet location.
Sehr sch��nes Hotel auf dem Lido.
Sehr sch��nes Hotel auf dem Lido.
I would come back
I would come back
une belle semaine �� Venise
Excellent Service
stayed before

In [ ]:
df = df.drop(A, axis = 0)

In [ ]:
df.groupby(["address"]).size()

address
1055 E Mcnee Road Of Main St                 15
111 N Post Oak Ln                            64
1118 Government St                           37
142 Park Place Us 181 Business At Park Pl     1
15249 N 59th Ave                              1
1700 N Livermore Ave                         85
1932 Lima Sandusky Rd                         1
2113 Sapp Brothers Dr                         1
2131 Kalakaua Ave                            21
23525 N E Halsey St                           1
2700 S Country Club Rd                       31
2860 Grand Ave                                1
3033 N Division St                            2
316 N Main St                                 2
375 Main St                                  31
435 E Avenida Olancha                        21
5820 Walden Rd                                1
864                                          62
Ostseeallee 10 12                            27
Po Box 1327                                  12
Riviera San Nicol 11/a          

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 4.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617556 sha256=b1077a8715a73242ddda4f0c0215fb342d52832eb23482b8aa0f084ee015ac58
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

In [ ]:
df["reviews.text"]

0      Pleasant 10 min walk along the sea front to th...
1      Really lovely hotel. Stayed on the very top fl...
3      We stayed here for four nights in October. The...
5      We loved staying on the island of Lido! You ne...
6      Lovely view out onto the lagoon. Excellent vie...
                             ...                        
494    It was an overnighter to get away from some stuff
495    Stayed overnight for pleasure. Room was clean ...
496    Ok overall - we stayed one night with our dogs...
497    I just ran up to the middle of Maine for a one...
498    Rooms have been updated and are very nice.  Po...
Name: reviews.text, Length: 456, dtype: object

In [ ]:
B = ["address","categories",	"city",	"country", "name",	"province",	"reviews.text",	"reviews.title"	,"reviews.userCity"	,"reviews.username",	"reviews.userProvince"]

for val in B:
    df[val] = df[val].astype("str")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 498
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   address               456 non-null    object
 1   categories            456 non-null    object
 2   city                  456 non-null    object
 3   country               456 non-null    object
 4   name                  456 non-null    object
 5   postalCode            456 non-null    int64 
 6   province              456 non-null    object
 7   reviews.rating        456 non-null    int64 
 8   reviews.text          456 non-null    object
 9   reviews.title         456 non-null    object
 10  reviews.userCity      456 non-null    object
 11  reviews.username      456 non-null    object
 12  reviews.userProvince  456 non-null    object
dtypes: int64(2), object(11)
memory usage: 49.9+ KB


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [ ]:
def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [ ]:
df['reviews.text'] = df['reviews.text'].apply(clean_txt)

In [ ]:
df['reviews.title'] = df['reviews.title'].apply(clean_txt)


In [ ]:
df

,address,categories,city,country,name,postalCode,province,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,4,pleasant min walk along sea front water bus re...,good location away crouds,Other,Russ (kent),Other
1,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,really lovely hotel stay top floor surprise ja...,great hotel jacuzzi bath,Other,A Traveler,Other
3,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,stay four nights october hotel staff welcome f...,good location lido,Other,Julie,Other
5,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,5,love stay island lido need take water venice g...,nice hotel,Other,A Traveler,Other
6,Riviera San Nicol 11/a,Hotels,Mableton,US,Hotel Russo Palace,30126,GA,4,lovely view onto lagoon excellent view staff w...,lovely view onto lagoon excellent view,Other,A Traveler,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,375 Main St,"Hotels,Hotel",Waterville,US,Best Western Plus Waterville Grand Hotel,4901,ME,4,overnighter get away stuff,good need better wifi,Belmont NH,Robert,New York
495,375 Main St,"Hotels,Hotel",Waterville,US,Best Western Plus Waterville Grand Hotel,4901,ME,4,stay overnight pleasure room clean well equip ...,good need better wifi,Belmont NH,A verified traveler,New York
496,375 Main St,"Hotels,Hotel",Waterville,US,Best Western Plus Waterville Grand Hotel,4901,ME,2,overall stay one night dog limit choices hotel...,need pet friendly hotel,Belmont NH,A Traveler,New York
497,375 Main St,"Hotels,Hotel",Waterville,US,Best Western Plus Waterville Grand Hotel,4901,ME,4,run middle maine one day business trip people ...,pleasant experience maine,Sag Harbor,James D,New York


In [ ]:
df_fake = df.drop(["country"], axis = 1)

In [ ]:
df_fake

,address,categories,city,name,postalCode,province,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,Hotel Russo Palace,30126,GA,4,pleasant min walk along sea front water bus re...,good location away crouds,Other,Russ (kent),Other
1,Riviera San Nicol 11/a,Hotels,Mableton,Hotel Russo Palace,30126,GA,5,really lovely hotel stay top floor surprise ja...,great hotel jacuzzi bath,Other,A Traveler,Other
3,Riviera San Nicol 11/a,Hotels,Mableton,Hotel Russo Palace,30126,GA,5,stay four nights october hotel staff welcome f...,good location lido,Other,Julie,Other
5,Riviera San Nicol 11/a,Hotels,Mableton,Hotel Russo Palace,30126,GA,5,love stay island lido need take water venice g...,nice hotel,Other,A Traveler,Other
6,Riviera San Nicol 11/a,Hotels,Mableton,Hotel Russo Palace,30126,GA,4,lovely view onto lagoon excellent view staff w...,lovely view onto lagoon excellent view,Other,A Traveler,Other
...,...,...,...,...,...,...,...,...,...,...,...,...
494,375 Main St,"Hotels,Hotel",Waterville,Best Western Plus Waterville Grand Hotel,4901,ME,4,overnighter get away stuff,good need better wifi,Belmont NH,Robert,New York
495,375 Main St,"Hotels,Hotel",Waterville,Best Western Plus Waterville Grand Hotel,4901,ME,4,stay overnight pleasure room clean well equip ...,good need better wifi,Belmont NH,A verified traveler,New York
496,375 Main St,"Hotels,Hotel",Waterville,Best Western Plus Waterville Grand Hotel,4901,ME,2,overall stay one night dog limit choices hotel...,need pet friendly hotel,Belmont NH,A Traveler,New York
497,375 Main St,"Hotels,Hotel",Waterville,Best Western Plus Waterville Grand Hotel,4901,ME,4,run middle maine one day business trip people ...,pleasant experience maine,Sag Harbor,James D,New York


In [ ]:
indexes = df_fake["name"].value_counts().index
values = df_fake["name"].value_counts().values

In [ ]:
map = {}
for i in range(20):
  map[indexes[i]] =  values[i]

In [ ]:
map

{'Americas Best Value Inn': 1,
 'Berney Fly Bed and Breakfast': 37,
 'Best Western Plus Waterville Grand Hotel': 31,
 'Comfort Suites': 15,
 'Days Inn El Reno Ok': 31,
 'Fairfield Inn By Marriott Binghamton': 62,
 'Gresham Area Hotels': 1,
 'Hawthorn Suites By Wyndham Livermore Wine Country': 85,
 'Hotel Russo Palace': 39,
 'Hotel Vier Jahreszeiten KÜhlungsborn': 27,
 'Howard Johnson': 3,
 'Intermezzo': 1,
 'La Quinta': 1,
 'Little Belt Inn Neihart': 2,
 'Little Paradise Hotel': 21,
 'Polynesian Plaza': 21,
 'Studio 6': 1,
 'The Houstonian Hotel': 64,
 'The Point': 12,
 'Thunderbird Executive Inn and Conference Center': 1}

In [ ]:
df_fake['reviews.text'] = " " + df_fake['reviews.text']

In [ ]:
df_fake['reviews.title'] = " " + df_fake['reviews.title']

In [ ]:
aggregation_functions = {'reviews.rating': 'mean', 
                         'reviews.text': 'sum', 
                         'reviews.title': 'sum',
                         'name': 'first', 
                         'postalCode': 'first',
                         'province': 'first', 
                         'address': 'first', 
                         'city': 'first', 
                         'categories':'first'}
df_new = df_fake.groupby(df_fake['name']).aggregate(aggregation_functions)

In [ ]:
df_new.groupby(["name"], axis = 1).size()

Series([], dtype: int64)

In [ ]:
df_new.index = [i for i in range(0, 20 )]

In [ ]:
df_new["count"] = [map[i] for i in df_new["name"]]

In [ ]:
df_new["reviews.text"].loc[0]

' share opinion businesswith visitors across unite statesand neighborhood'

In [ ]:
df_new["reviews.text"] = df_new["reviews.text"][1:]

In [ ]:
df_new["reviews.title"] = df_new["reviews.title"][1:]

In [ ]:
df_new

,reviews.rating,reviews.text,reviews.title,name,postalCode,province,address,city,categories,count
0,0.000000,NaN,NaN,Americas Best Value Inn,51648,IA,2113 Sapp Brothers Dr,Percival,"Lodging,Hotels,Motels",1
1,4.540541,stay poolside room one night love backyard po...,comfortable safe pleasant garden wifi poolsid...,Berney Fly Bed and Breakfast,36604,AL,1118 Government St,Mobile,"Bed & Breakfasts,Hotels,Hotels & Motels,Lodgin...",37
2,3.806452,ask nonsmoking room please fact didnt seem li...,better expect easy access interstate yet stil...,Best Western Plus Waterville Grand Hotel,4901,ME,375 Main St,Waterville,"Hotels,Hotel",31
3,3.666667,treat texas medical center facility good plac...,home away home discriminatory great place sta...,Comfort Suites,77054,TX,1055 E Mcnee Road Of Main St,Houston,"Corporate Lodging,Hotel & Motel Reservations,H...",15
4,2.967742,room look disrepair bathroom door didnt shut ...,hotels bad extend stay hotels bad extend stay...,Days Inn El Reno Ok,73036,OK,2700 S Country Club Rd,El Reno,Hotels,31
5,4.096774,come binghamton visit relative fairfield inn ...,would stay dont stay good stay nice hotel c...,Fairfield Inn By Marriott Binghamton,13905,Choconut Center,864,Binghamton,Hotels,62
6,0.000000,share opinion businesswith visitors across un...,,Gresham Area Hotels,97060,OR,23525 N E Halsey St,Troutdale,"Hotels,Lodging",1
7,3.929412,room fine mattress comfortable unfortunately ...,nice room dirty room happy happy wine lovely ...,Hawthorn Suites By Wyndham Livermore Wine Country,94551,CA,1700 N Livermore Ave,Livermore,Hotels,85
8,3.974359,pleasant min walk along sea front water bus r...,good location away crouds great hotel jacuzzi...,Hotel Russo Palace,30126,GA,Riviera San Nicol 11/a,Mableton,Hotels,39
9,4.111111,haupthaus kein passendes zimmer mehr frei war...,hotel sehr guter lage great hotel relax zentr...,Hotel Vier Jahreszeiten KÜhlungsborn,18225,PA,Ostseeallee 10 12,Harleigh,"Gastronomic specialties,Meat,Foods, precooked ...",27


In [ ]:
df_content = df_new

In [ ]:
df_collab = df[['reviews.rating', 'name', 'reviews.username']]

In [ ]:
df_content

,reviews.rating,reviews.text,reviews.title,name,postalCode,province,address,city,categories,count
0,0.000000,NaN,NaN,Americas Best Value Inn,51648,IA,2113 Sapp Brothers Dr,Percival,"Lodging,Hotels,Motels",1
1,4.540541,stay poolside room one night love backyard po...,comfortable safe pleasant garden wifi poolsid...,Berney Fly Bed and Breakfast,36604,AL,1118 Government St,Mobile,"Bed & Breakfasts,Hotels,Hotels & Motels,Lodgin...",37
2,3.806452,ask nonsmoking room please fact didnt seem li...,better expect easy access interstate yet stil...,Best Western Plus Waterville Grand Hotel,4901,ME,375 Main St,Waterville,"Hotels,Hotel",31
3,3.666667,treat texas medical center facility good plac...,home away home discriminatory great place sta...,Comfort Suites,77054,TX,1055 E Mcnee Road Of Main St,Houston,"Corporate Lodging,Hotel & Motel Reservations,H...",15
4,2.967742,room look disrepair bathroom door didnt shut ...,hotels bad extend stay hotels bad extend stay...,Days Inn El Reno Ok,73036,OK,2700 S Country Club Rd,El Reno,Hotels,31
5,4.096774,come binghamton visit relative fairfield inn ...,would stay dont stay good stay nice hotel c...,Fairfield Inn By Marriott Binghamton,13905,Choconut Center,864,Binghamton,Hotels,62
6,0.000000,share opinion businesswith visitors across un...,,Gresham Area Hotels,97060,OR,23525 N E Halsey St,Troutdale,"Hotels,Lodging",1
7,3.929412,room fine mattress comfortable unfortunately ...,nice room dirty room happy happy wine lovely ...,Hawthorn Suites By Wyndham Livermore Wine Country,94551,CA,1700 N Livermore Ave,Livermore,Hotels,85
8,3.974359,pleasant min walk along sea front water bus r...,good location away crouds great hotel jacuzzi...,Hotel Russo Palace,30126,GA,Riviera San Nicol 11/a,Mableton,Hotels,39
9,4.111111,haupthaus kein passendes zimmer mehr frei war...,hotel sehr guter lage great hotel relax zentr...,Hotel Vier Jahreszeiten KÜhlungsborn,18225,PA,Ostseeallee 10 12,Harleigh,"Gastronomic specialties,Meat,Foods, precooked ...",27


In [ ]:
df_content["overview"] = df_content["reviews.text"] + df_content["reviews.title"] +  df_content["categories"] + df_content["province"] + df_content["city"] + df_content["address"]

In [ ]:
df_content

,reviews.rating,reviews.text,reviews.title,name,postalCode,province,address,city,categories,count,overview
0,0.000000,NaN,NaN,Americas Best Value Inn,51648,IA,2113 Sapp Brothers Dr,Percival,"Lodging,Hotels,Motels",1,NaN
1,4.540541,stay poolside room one night love backyard po...,comfortable safe pleasant garden wifi poolsid...,Berney Fly Bed and Breakfast,36604,AL,1118 Government St,Mobile,"Bed & Breakfasts,Hotels,Hotels & Motels,Lodgin...",37,stay poolside room one night love backyard po...
2,3.806452,ask nonsmoking room please fact didnt seem li...,better expect easy access interstate yet stil...,Best Western Plus Waterville Grand Hotel,4901,ME,375 Main St,Waterville,"Hotels,Hotel",31,ask nonsmoking room please fact didnt seem li...
3,3.666667,treat texas medical center facility good plac...,home away home discriminatory great place sta...,Comfort Suites,77054,TX,1055 E Mcnee Road Of Main St,Houston,"Corporate Lodging,Hotel & Motel Reservations,H...",15,treat texas medical center facility good plac...
4,2.967742,room look disrepair bathroom door didnt shut ...,hotels bad extend stay hotels bad extend stay...,Days Inn El Reno Ok,73036,OK,2700 S Country Club Rd,El Reno,Hotels,31,room look disrepair bathroom door didnt shut ...
5,4.096774,come binghamton visit relative fairfield inn ...,would stay dont stay good stay nice hotel c...,Fairfield Inn By Marriott Binghamton,13905,Choconut Center,864,Binghamton,Hotels,62,come binghamton visit relative fairfield inn ...
6,0.000000,share opinion businesswith visitors across un...,,Gresham Area Hotels,97060,OR,23525 N E Halsey St,Troutdale,"Hotels,Lodging",1,share opinion businesswith visitors across un...
7,3.929412,room fine mattress comfortable unfortunately ...,nice room dirty room happy happy wine lovely ...,Hawthorn Suites By Wyndham Livermore Wine Country,94551,CA,1700 N Livermore Ave,Livermore,Hotels,85,room fine mattress comfortable unfortunately ...
8,3.974359,pleasant min walk along sea front water bus r...,good location away crouds great hotel jacuzzi...,Hotel Russo Palace,30126,GA,Riviera San Nicol 11/a,Mableton,Hotels,39,pleasant min walk along sea front water bus r...
9,4.111111,haupthaus kein passendes zimmer mehr frei war...,hotel sehr guter lage great hotel relax zentr...,Hotel Vier Jahreszeiten KÜhlungsborn,18225,PA,Ostseeallee 10 12,Harleigh,"Gastronomic specialties,Meat,Foods, precooked ...",27,haupthaus kein passendes zimmer mehr frei war...


In [ ]:
df_collab = df_collab.drop(df_collab[df_collab['name'] == "Americas Best Value Inn"].index)

In [ ]:
df = df.drop(df[df['name'] == "Americas Best Value Inn"].index)

In [ ]:
df_collab

,reviews.rating,name,reviews.username
0,4,Hotel Russo Palace,Russ (kent)
1,5,Hotel Russo Palace,A Traveler
3,5,Hotel Russo Palace,Julie
5,5,Hotel Russo Palace,A Traveler
6,4,Hotel Russo Palace,A Traveler
...,...,...,...
494,4,Best Western Plus Waterville Grand Hotel,Robert
495,4,Best Western Plus Waterville Grand Hotel,A verified traveler
496,2,Best Western Plus Waterville Grand Hotel,A Traveler
497,4,Best Western Plus Waterville Grand Hotel,James D


In [ ]:
df_content = df_content.dropna()

In [ ]:
tf_2 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_2 = tf_2.fit_transform(df_content['overview'])

In [ ]:
import pickle
pickle.dump(tfidf_matrix_2, open('tfidf_jobid.pickle', 'wb'))
pickle.dump(tf_2, open('tfidf_vectorizer.pickle', 'wb'))

In [ ]:
cosine_sim_2 = cosine_similarity(tfidf_matrix_2, tfidf_matrix_2)

In [ ]:
df_content = df_content.reset_index()
titles = df_content['name']
indices = pd.Series(df_content.index, index=df_content['name'])

In [ ]:
df_ttt = pd.read_csv("/content/huhu.csv")

In [ ]:
df_ttt = df_ttt.drop(['Unnamed: 0', 'index'], axis = 1)


In [ ]:
df_ttt

,reviews.rating,reviews.text,reviews.title,name,postalCode,province,address,city,categories,count,overview
0,4.540541,stay poolside room one night love backyard po...,comfortable safe pleasant garden wifi poolsid...,Berney Fly Bed and Breakfast,36604,AL,1118 Government St,Mobile,"Bed & Breakfasts,Hotels,Hotels & Motels,Lodgin...",37,stay poolside room one night love backyard po...
1,3.806452,ask nonsmoking room please fact didnt seem li...,better expect easy access interstate yet stil...,Best Western Plus Waterville Grand Hotel,4901,ME,375 Main St,Waterville,"Hotels,Hotel",31,ask nonsmoking room please fact didnt seem li...
2,3.666667,treat texas medical center facility good plac...,home away home discriminatory great place sta...,Comfort Suites,77054,TX,1055 E Mcnee Road Of Main St,Houston,"Corporate Lodging,Hotel & Motel Reservations,H...",15,treat texas medical center facility good plac...
3,2.967742,room look disrepair bathroom door didnt shut ...,hotels bad extend stay hotels bad extend stay...,Days Inn El Reno Ok,73036,OK,2700 S Country Club Rd,El Reno,Hotels,31,room look disrepair bathroom door didnt shut ...
4,4.096774,come binghamton visit relative fairfield inn ...,would stay dont stay good stay nice hotel c...,Fairfield Inn By Marriott Binghamton,13905,Choconut Center,864,Binghamton,Hotels,62,come binghamton visit relative fairfield inn ...
5,0.000000,share opinion businesswith visitors across un...,,Gresham Area Hotels,97060,OR,23525 N E Halsey St,Troutdale,"Hotels,Lodging",1,share opinion businesswith visitors across un...
6,3.929412,room fine mattress comfortable unfortunately ...,nice room dirty room happy happy wine lovely ...,Hawthorn Suites By Wyndham Livermore Wine Country,94551,CA,1700 N Livermore Ave,Livermore,Hotels,85,room fine mattress comfortable unfortunately ...
7,3.974359,pleasant min walk along sea front water bus r...,good location away crouds great hotel jacuzzi...,Hotel Russo Palace,30126,GA,Riviera San Nicol 11/a,Mableton,Hotels,39,pleasant min walk along sea front water bus r...
8,4.111111,haupthaus kein passendes zimmer mehr frei war...,hotel sehr guter lage great hotel relax zentr...,Hotel Vier Jahreszeiten KÜhlungsborn,18225,PA,Ostseeallee 10 12,Harleigh,"Gastronomic specialties,Meat,Foods, precooked ...",27,haupthaus kein passendes zimmer mehr frei war...
9,2.666667,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx fa...,great place great place,Howard Johnson,44870,Bloomingville,1932 Lima Sandusky Rd,Sandusky,"Lodging,Corporate Lodging,Hotels",3,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx fa...


In [ ]:
df_content

,index,reviews.rating,reviews.text,reviews.title,name,postalCode,province,address,city,categories,count,overview
0,1,4.540541,stay poolside room one night love backyard po...,comfortable safe pleasant garden wifi poolsid...,Berney Fly Bed and Breakfast,36604,AL,1118 Government St,Mobile,"Bed & Breakfasts,Hotels,Hotels & Motels,Lodgin...",37,stay poolside room one night love backyard po...
1,2,3.806452,ask nonsmoking room please fact didnt seem li...,better expect easy access interstate yet stil...,Best Western Plus Waterville Grand Hotel,4901,ME,375 Main St,Waterville,"Hotels,Hotel",31,ask nonsmoking room please fact didnt seem li...
2,3,3.666667,treat texas medical center facility good plac...,home away home discriminatory great place sta...,Comfort Suites,77054,TX,1055 E Mcnee Road Of Main St,Houston,"Corporate Lodging,Hotel & Motel Reservations,H...",15,treat texas medical center facility good plac...
3,4,2.967742,room look disrepair bathroom door didnt shut ...,hotels bad extend stay hotels bad extend stay...,Days Inn El Reno Ok,73036,OK,2700 S Country Club Rd,El Reno,Hotels,31,room look disrepair bathroom door didnt shut ...
4,5,4.096774,come binghamton visit relative fairfield inn ...,would stay dont stay good stay nice hotel c...,Fairfield Inn By Marriott Binghamton,13905,Choconut Center,864,Binghamton,Hotels,62,come binghamton visit relative fairfield inn ...
5,6,0.000000,share opinion businesswith visitors across un...,,Gresham Area Hotels,97060,OR,23525 N E Halsey St,Troutdale,"Hotels,Lodging",1,share opinion businesswith visitors across un...
6,7,3.929412,room fine mattress comfortable unfortunately ...,nice room dirty room happy happy wine lovely ...,Hawthorn Suites By Wyndham Livermore Wine Country,94551,CA,1700 N Livermore Ave,Livermore,Hotels,85,room fine mattress comfortable unfortunately ...
7,8,3.974359,pleasant min walk along sea front water bus r...,good location away crouds great hotel jacuzzi...,Hotel Russo Palace,30126,GA,Riviera San Nicol 11/a,Mableton,Hotels,39,pleasant min walk along sea front water bus r...
8,9,4.111111,haupthaus kein passendes zimmer mehr frei war...,hotel sehr guter lage great hotel relax zentr...,Hotel Vier Jahreszeiten KÜhlungsborn,18225,PA,Ostseeallee 10 12,Harleigh,"Gastronomic specialties,Meat,Foods, precooked ...",27,haupthaus kein passendes zimmer mehr frei war...
9,10,2.666667,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx fa...,great place great place,Howard Johnson,44870,Bloomingville,1932 Lima Sandusky Rd,Sandusky,"Lodging,Corporate Lodging,Hotels",3,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx fa...


In [ ]:
df_content.to_csv("huhu.csv")

In [ ]:
def improved_recommendations_2(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_2[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
def weighted_rating(x):
    v = x['count']
    R = x['reviews.rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
title = "Berney Fly Bed and Breakfast"
idx = indices[title]
sim_scores = list(enumerate(cosine_sim_2[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:26]
movie_indices = [i[0] for i in sim_scores]
  

In [ ]:
movies = df_content.iloc[movie_indices][['name', 'count', 'reviews.rating']]
vote_counts = movies[movies['count'].notnull()]['count'].astype('int')
vote_averages = movies[movies['reviews.rating'].notnull()]['reviews.rating'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.60)
qualified = movies[(movies['count'] >= m) & (movies['count'].notnull()) & (movies['reviews.rating'].notnull())]
qualified['count'] = qualified['count'].astype('int')
qualified['reviews.rating'] = qualified['reviews.rating'].astype('int')
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(10)

In [ ]:
qualified

,name,count,reviews.rating,wr
16,The Houstonian Hotel,64,4,3.585073
4,Fairfield Inn By Marriott Binghamton,62,4,3.575218
8,Hotel Vier Jahreszeiten KÜhlungsborn,27,4,3.273035
6,Hawthorn Suites By Wyndham Livermore Wine Country,85,3,2.873445
7,Hotel Russo Palace,39,3,2.778322
1,Best Western Plus Waterville Grand Hotel,31,3,2.744987
3,Days Inn El Reno Ok,31,2,2.162281


In [ ]:
df_collab.shape

(455, 3)

In [ ]:
df_collab

,reviews.rating,name,reviews.username
0,4,Hotel Russo Palace,Russ (kent)
1,5,Hotel Russo Palace,A Traveler
3,5,Hotel Russo Palace,Julie
5,5,Hotel Russo Palace,A Traveler
6,4,Hotel Russo Palace,A Traveler
...,...,...,...
494,4,Best Western Plus Waterville Grand Hotel,Robert
495,4,Best Western Plus Waterville Grand Hotel,A verified traveler
496,2,Best Western Plus Waterville Grand Hotel,A Traveler
497,4,Best Western Plus Waterville Grand Hotel,James D


In [ ]:
df_collab.groupby(["reviews.username"]).size()["A Traveler"]

105

In [ ]:
df_collab.index = [i for i in range(0, 455)]

In [ ]:
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 0 to 454
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reviews.rating    455 non-null    int64 
 1   name              455 non-null    object
 2   reviews.username  455 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.2+ KB
